In [ ]:
#use this if using colab or other compilers need files in drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Import necessary libraries
!pip install PyPDF2
!pip install uszipcode
import os
from PyPDF2 import PdfReader
import re
import pandas as pd

# Directory containing all your PDF files in a drive folder
pdf_folder = '/content/drive/My Drive/new files/'

# List all PDF files in the folder
pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]

# Initialize a list to store captured texts from all PDFs
captured_texts_all_pdfs = []

# Specify multiple start and end words for text extraction
start_words = ["Requested By", "Customer Name :", "Phone", "Cell", "FL Address :", "Work Dates",
               "Description of Services Completed:", "Service Representative"]
end_words = ["Phone", "FL", "Date Received", ",", "Bill To", "Contract No.", "Recommendations/Comments", "Hours"]

# Initialize an empty DataFrame with the desired column names
column_names = ["Sl. No", "Contact Name", "Company", "Phone", "Cell", "Location", "Service Date",
                "Description", "Service Representative"]

#intitialize a dataframe that contains your columns
df = pd.DataFrame(columns=column_names)

# Loop through each PDF file
for serial_no, pdf_file in enumerate(pdf_files, start=1):
    pdf_path = os.path.join(pdf_folder, pdf_file)
    reader = PdfReader(pdf_path)

    # Initialize text variable for this PDF file to store the text
    text = ""

    # Extract text from all pages and concatenate
    for page in reader.pages:
        text += page.extract_text()

    # Initialize a dictionary to store captured text for this PDF
    captured_texts = {column_name: "" for column_name in column_names}

    # look for alternate key words for "Description" column
    start_word = "Description of Services Completed:"
    end_word = "Recommendations/Comments"

    start_index = text.find(start_word)
    end_index = text.find(end_word)

    # If initial start and end words are not found, use alternate words
    if start_index == -1 or end_index == -1:
        start_word = "Work Description/ Customer Benefits/ Recommendations"
        end_word = "System Restored Date"
        start_index = text.find(start_word)
        end_index = text.find(end_word)


    if start_index != -1 and end_index != -1:
        captured_text = text[start_index + len(start_word):end_index].strip()
        # Remove newline characters from the captured text using regular expression
        captured_text = re.sub(r'\n', ' ', captured_text)
        # Store the captured text in the dictionary
        captured_texts["Description"] = captured_text


    # Loop through each pair of start and end words (excluding "Description")
    for start_word, end_word, column_name in zip(start_words, end_words, column_names[1:]):  # Exclude "Serial No."
        index = 0
        while True:
            start_index = text.find(start_word, index)
            end_index = text.find(end_word, index)

            if start_index != -1 and end_index != -1:
                captured_text = text[start_index + len(start_word):end_index].strip()
                # Remove newline characters from the captured text using regular expression
                captured_text = re.sub(r'\n', ' ', captured_text)
                # Store the captured text in the dictionary
                captured_texts[column_name] = captured_text
                index = end_index + len(end_word)
            else:
                break

    # Add the Serial No. to the dictionary
    captured_texts["Sl. No"] = serial_no

    # Append the captured texts for this PDF to the list for all PDFs
    captured_texts_all_pdfs.append(captured_texts)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(captured_texts_all_pdfs)


In [ ]:
df

In [ ]:
from uszipcode import SearchEngine

# Create a search engine instance
search = SearchEngine()

# Define a range of possible ZIP codes (adjust as needed)
start_zipcode = "00000"
end_zipcode = "99999"

# Generate a list of valid ZIP codes within the specified range
valid_zipcodes = [str(zipcode) for zipcode in range(int(start_zipcode), int(end_zipcode) + 1)]

# Also generate a list of valid 5-digit ZIP codes without the area code
valid_zipcodes_without_area_code = [zipcode[:-4] for zipcode in valid_zipcodes]

# Combine both lists of valid ZIP codes
valid_zipcodes += valid_zipcodes_without_area_code

# Remove duplicates by converting the list to a set and back to a list
valid_zipcodes = list(set(valid_zipcodes))

# Function to extract and standardize location based on ZIP code
def extract_location(location):
    # Check if the location ends with a pattern like "-nnnn"
    if re.search(r'-\d{4}$', location):
        # If the pattern is found, remove it
        location = re.sub(r'-\d{4}$', '', location)

    # Iterate through valid ZIP codes
    for zipcode in valid_zipcodes:
        if location.endswith(zipcode):
            # Attempt to lookup ZIP code information
            zipcode_info = search.by_zipcode(zipcode)

            # Check if the lookup was successful and contains valid information
            if zipcode_info and zipcode_info.major_city and zipcode_info.state:
                return f"{zipcode_info.major_city}, {zipcode_info.state} {zipcode}"

    # If no matching ZIP code is found or the lookup fails, return the location as is
    return location

# Apply the extract_location function to the "Location" column
df["Location"] = df["Location"].apply(extract_location)


Exception ignored in: <function SearchEngine.__del__ at 0x7de3c69ffc70>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uszipcode/search.py", line 196, in __del__
  File "/usr/local/lib/python3.10/dist-packages/uszipcode/search.py", line 202, in close
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/orm/session.py", line 1816, in close
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/orm/session.py", line 1858, in _close_impl
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/orm/session.py", line 926, in close
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/engine/base.py", line 2426, in close
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/engine/base.py", line 2649, in _do_close
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/engine/base.py", line 2635, in _close_impl
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/engine/base.py", line 2627, in _connection_rollback_impl
  File

In [ ]:
df

In [ ]:
# Convert 'Contact Name' to title case (capitalize first letter of each word)
df['Contact Name'] = df['Contact Name'].str.title()

# Convert 'Company' to uppercase
df['Company'] = df['Company'].str.upper()

# Assuming the phone numbers are initially in different formats:
# 1. Remove all non-numeric characters from 'Phone'
# 2. Format 'Phone' as XXX-XXX-XXXX
df['Phone'] = df['Phone'].str.replace(r'[^0-9]', '', regex=True)
df['Phone'] = df['Phone'].str.replace(r'(\d{3})(\d{3})(\d{4})', r'\1-\2-\3', regex=True)

# Remove trailing special characters from the 'Description' column
df['Description'] = df['Description'].str.rstrip('[^a-zA-Z0-9\s]')

# Remove all numerical content from the 'FL Address' (Location) column
df['Location'] = df['Location'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

# Remove non-alphanumeric characters from 'Service Representative'
df['Service Representative'] = df['Service Representative'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)


In [ ]:
df

In [ ]:
df.to_csv('siemensreportforseptmber.csv', index=False)